In [1]:
import pandas as pd

In [17]:
in_file = "/Users/datascience/Desktop/WPDX Plus - All Regions - Piped Water.csv"
# in_file = "/Users/datascience/Desktop/wpdx-ghana-plusdata.csv"

In [18]:
df = pd.read_csv(in_file)
df.shape

(2524, 73)

In [19]:
df.head(3)

,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source_clean,#water_source_category,#water_tech_clean,#water_tech_category,...,#subjective_quality,New Georeferenced Column,lat_lon_deg,public_data_source,converted,count,created_timestamp,updated_timestamp,is_duplicate,dataset_title
0,"369,599",Water4,5.233200,-1.447812,06/05/2023 12:00:00 AM,Yes,Piped Water,Piped Water,Public Tapstand,Public Tapstand,...,NaN,POINT (-1.4478117 5.233199999999999),"(5.233199999999999°, -1.4478117°)",https://catalog.waterpointdata.org/datafiles/w...,NaN,1,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,True,Water4_Global_2021-2022
1,"282,781",Water4,5.452758,-1.717567,12/14/2022 12:00:00 AM,Yes,Piped Water,Piped Water,Public Tapstand,Public Tapstand,...,NaN,POINT (-1.7175667 5.452758299999999),"(5.452758299999999°, -1.7175667°)",https://catalog.waterpointdata.org/datafiles/w...,NaN,1,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,True,Water4_Global_2021-2022
2,"395,199",Water4,9.492330,-2.437187,11/16/2023 12:00:00 AM,Yes,Piped Water,Piped Water,Public Tapstand,Public Tapstand,...,NaN,POINT (-2.4371871999999994 9.492330200000001),"(9.492330200000001°, -2.4371871999999994°)",https://catalog.waterpointdata.org/datafiles/w...,NaN,1,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,NaN,Water4_Global_2021-2022


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2524 entries, 0 to 2523
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   row_id                       2524 non-null   object 
 1   #source                      2524 non-null   object 
 2   #lat_deg                     2524 non-null   float64
 3   #lon_deg                     2524 non-null   float64
 4   #report_date                 2524 non-null   object 
 5   #status_id                   2524 non-null   object 
 6   #water_source_clean          2524 non-null   object 
 7   #water_source_category       2524 non-null   object 
 8   #water_tech_clean            2051 non-null   object 
 9   #water_tech_category         2051 non-null   object 
 10  #facility_type               2524 non-null   object 
 11  #clean_country_name          2524 non-null   object 
 12  #clean_country_id            2524 non-null   object 
 13  #clean_adm1       

In [21]:
df["#water_source_clean"].value_counts()

#water_source_clean
Piped Water    2524
Name: count, dtype: int64

In [22]:
df["#clean_adm1"].value_counts()

#clean_adm1
Upper West    1152
Western        818
Savannah       409
Central        108
Oti             30
Northern         3
Ashanti          2
Upper East       2
Name: count, dtype: int64

In [9]:
df["#source"].value_counts()

#source
Water4                        2238
Water4 Headquarters            236
YouthMappers Ghana Chapter      30
Water4 Public                   14
WaterAid                         5
WaterAid UK                      1
Name: count, dtype: int64

In [10]:
df[df["#clean_adm1"].isin(["Ashanti", "Eastern", "Volta"])]

,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source_clean,#water_source_category,#water_tech_clean,#water_tech_category,...,#subjective_quality,New Georeferenced Column,lat_lon_deg,public_data_source,converted,count,created_timestamp,updated_timestamp,is_duplicate,dataset_title
46,"143,211",Water4,6.451265,-1.310293,06/23/2022 12:00:00 AM,Yes,Piped Water,Piped Water,Public Tapstand,Public Tapstand,...,NaN,POINT (-1.310293121 6.451264867),"(6.451264867°, -1.310293121°)",https://catalog.waterpointdata.org/datafiles/w...,NaN,1,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,True,Water4_Global_2021-2022
212,"142,659",Water4,7.235403,-1.578426,06/16/2022 12:00:00 AM,Yes,Piped Water,Piped Water,Public Tapstand,Public Tapstand,...,NaN,POINT (-1.5784263609999998 7.235402627999998),"(7.235402627999998°, -1.5784263609999998°)",https://catalog.waterpointdata.org/datafiles/w...,NaN,1,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,True,Water4_Global_2021-2022


In [11]:
df[df["#clean_adm1"].isin(["Bono", "Ahafo"])]

,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source_clean,#water_source_category,#water_tech_clean,#water_tech_category,...,#subjective_quality,New Georeferenced Column,lat_lon_deg,public_data_source,converted,count,created_timestamp,updated_timestamp,is_duplicate,dataset_title
